In [ ]:
#!usr/bin/env python
#-*- coding: utf-8 -*-

# First XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

import pandas
pandas.__version__

# load data
dataset = loadtxt("C:\\Dev\\TSS\\xgboost_test\\pima-indians-diabetes.csv", delimiter=",")
# split data into X and y
X = dataset[:,:8]
Y = dataset[:,8]
# split data into train and test sets
seed = 7
test_size = 0.33
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=test_size, random_state=seed)

print(X_test[0])

# fit model on training data
model = XGBClassifier()

eval_set = [(X_test, y_test)]

model.fit(X_train, y_train)

#model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="logloss", eval_set=eval_set, verbose=True)

# make predictions for test data
y_predictions = model.predict(X_test)
# evaluate predictions
accuracy = accuracy_score(y_test, y_predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

#plot_importance(model)
#pyplot.show()


In [34]:
#!usr/bin/env python
#-*- coding: utf-8 -*-

# First XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold

# set common XGBoost parameters
# https://stackoverflow.com/questions/34674797/xgboost-xgbclassifier-defaults-in-python

# load data
dataset = loadtxt("C:\\Dev\\TSS\\xgboost_test\\pima-indians-diabetes.csv", delimiter=",")
# split data into X and y
X = dataset[:,0:8]
Y = dataset[:,8]
learning_rate = [0.0001, 0.001, 0.01, 0.1, 0.2, 0.3]
param_grid = dict(learning_rate=learning_rate)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=7)
grid_search = GridSearchCV(model, param_grid, scoring="neg_log_loss", n_jobs=-1, cv=kfold)
grid_result = grid_search.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
  print("%f (%f) with: %r" % (mean, stdev, param))

Best: -0.483304 using {'learning_rate': 0.1}
-0.689811 (0.000475) with: {'learning_rate': 0.0001}
-0.661827 (0.004625) with: {'learning_rate': 0.001}
-0.531155 (0.028945) with: {'learning_rate': 0.01}
-0.483304 (0.055151) with: {'learning_rate': 0.1}
-0.515642 (0.061723) with: {'learning_rate': 0.2}
-0.554158 (0.067557) with: {'learning_rate': 0.3}


In [26]:
# First XGBoost model for Pima Indians dataset
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from xgboost import plot_importance
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
import sys
import os
import time
import numpy as np
import pickle as pickle
import importlib 
import pandas as pd
from sklearn import metrics

import pandas
pandas.__version__

# set common XGBoost parameters
# https://stackoverflow.com/questions/34674797/xgboost-xgbclassifier-defaults-in-python

# load data
headers = ['CK_AGMT_BAL', 'CK_COMP_LAST_MON_BAL1', 'CK_COMP_LAST_MON_BAL2', 'CK_COMP_LAST_MON_BAL3', 'CK_YEAR_AVG_BAL', 'CK_COMP_LAST_MON_YEARAVG_BAL1', 'CK_COMP_LAST_MON_YEARAVG_BAL2', 'CK_COMP_LAST_MON_YEARAVG_BAL3', 'DK_AGMT_BAL', 'DK_COMP_LAST_MON_BAL1', 'DK_COMP_LAST_MON_BAL2', 'DK_COMP_LAST_MON_BAL3', 'DK_COMP_LAST_MON_YEARAVG_BAL1', 'DK_COMP_LAST_MON_YEARAVG_BAL2', 'DK_COMP_LAST_MON_YEARAVG_BAL3', 'FIN_BAL', 'FIN_BAL_SM', 'FIN_BAL_SQ', 'YEAR_FIN_BAL', 'YEAR_FIN_BAL_SM', 'YEAR_FIN_BAL_SQ', 'DF_AMT', 'DF_RT', 'DF_CNT', 'LST_1M_AMT', 'LST_1M_TT', 'LST_1M_CNT', 'LST_2M_AMT', 'LST_2M_TT', 'LST_2M_CNT', 'DK_SHUI', 'DK_DIAN', 'DK_MEI', 'IND_JBH', 'IND_WY', 'IND_YTZH', 'IND_YTCYL', 'IND_BGYDT', 'IND_JST', 'IND_DF', 'IND_CCARD', 'IND_FUND', 'IND_QS', 'IND_TXP', 'IND_BOND', 'IND_SX', 'IND_BX', 'IND_DLB', 'IND_SFCG', 'IND_YQT', 'IND_SJYH', 'IND_TALC', 'IND_GOLD', 'IND_JYT', 'IND_EJZH', 'IND_XJC', 'IND_YTDB', 'IND_PJC', 'IND_WBXJC', 'IND_LDXJC', 'IND_JZSF', 'IND_YQZTC', 'IND_YTSZ', 'IND_BZT', 'IND_ZBT', 'IND_JSK', 'IND_EJFH', 'IND_DLZY', 'IND_DLDF', 'IND_DLMK', 'IND_ZHQC', 'IND_DFGZ', 'IND_CKH', 'IND_HQCK', 'IND_XDCK', 'IND_DWHQ', 'IND_SG', 'IND_YQZZ', 'IND_RZRQ', 'REG_CAPT_AMT', 'A00', 'A01', 'A02', 'B00', 'B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B09', 'B10', 'B11', 'B12', 'B13', 'B14', 'B15', 'C00', 'C01', 'C02', 'C03', 'D00', 'D01', 'D02', 'D03', 'D04', 'E00', 'E01', 'E02', 'F00', 'F01', 'F02', 'F03', 'F04', 'F05', 'F06', 'F07', 'G00', 'G01', 'G02', 'G03', 'G04', 'G05', 'G06', 'M00', 'M01', 'M02', 'M03', 'M04', 'M05', 'M06', 'M07', 'Y01', 'Z00', 'Z01', 'INTER_RATING_CD', 'O_INDUSTRY_CD', 'S_INDUSTRY_CD', 'T_INDUSTRY_CD', 'F_INDUSTRY_CD', 'CUS_SCALE_TP_CD', 'ORG_TP_CD', 'INDUSTRY_TP_CD', 'INDUSTRY_MAIN_TP_CD', 'CUS_LAYER_CD', 'CUS_CLASS_CD', 'CUS_GRADE_CD', 'LMT_STATE_CD', 'CREDIT_LINE_CD', 'AERA', 'BAS_AC_IS_BOCOM', 'OWN_GROUP_LAYER', 'OWN_GROUP_CLASS', 'OWN_GROUP_GRADE', 'STRATEGIC_CUST', 'STRATEGIC_MGR_CODE', 'APPROVED_LMT', 'APPROVED_RISK_LMT', 'BAL', 'RISK_BAL', 'ORI_BAL', 'ZT_LMT', 'LXD_LMT', 'ENTSTATUS', 'IS_SHIXIN', 'IS_ANDI', 'IS_PERFORMANCE', 'IS_GL_SHIXIN', 'IS_GL_ANDI', 'IS_GL_PERFORMANCE', 'IS_YJ']
update_pd = pd.read_csv("C:\\Dev\TSS\\Bank_COMM\\risk_update_3.csv", sep = ',', names = headers)
    
test_pd = update_pd.fillna(0)

    
X_train, X_test, y_train, y_test = train_test_split(test_pd.iloc[:,1:172], test_pd['IS_YJ'], test_size=0.3, random_state=0)

# fit model on training data
model = XGBClassifier()

eval_set = [(X_test, y_test)]

model.fit(X_train, y_train)

#model.fit(X_train, y_train, early_stopping_rounds=10, eval_metric="logloss", eval_set=eval_set, verbose=True)

# make predictions for test data
y_predictions = model.predict(X_test)
# evaluate predictions
accuracy = metrics.accuracy_score(y_test, y_predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

acc = metrics.accuracy_score(y_test,y_predictions)
recall = metrics.recall_score(y_test,y_predictions)
f1 = metrics.f1_score(y_test,y_predictions)
precision = metrics.precision_score(y_test,y_predictions)

print("ACC: %.4f" % (acc * 100.0))

print("Precesion: %.4f" % (acc * 100.0))      
      
print("Recall: %.4f" % (recall * 100.0))
print("F1-score: %.4f" % (f1 * 100.0))it
metrics.confusion_matrix(y_test,y_predictions)

#plot_importance(model)
#pyplot.show()

Accuracy: 92.04%
ACC: 92.0429
Precesion: 92.0429
Recall: 10.4247
F1-score: 17.8512


C:\Dev\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[5695,   33],
       [ 464,   54]], dtype=int64)